Reference paper: stability is stable 
Experiment: vary the value of rho and see the minimum sample size required for replicability

func getConvergenceSampleNum(hyperparams)
    For sample_num range(min_subsets_size, max_subsets_size, step):
        For range(repeat_num):
            random draw a subset of the dataset, whose size = sample_num
            Use algorithm 10 to get a model
        Check whether the models we get are 'replicable' to each other, accoding to the hyperparams. (see definition in page 14 of the paper)
        if converged, return current sample_num

func experiment(): 
# vary the rho and see minumum sample size required for replicability. 
# To make it simple, we can fix the value of other hyperparamers, such as alpha and beta.
    for rho in range(min_rho, max_rho, step):
        sample_num = getConvergenceSampleNum(rho)
        theoretical_sample_num = getTheoreticalSampleNum(rho) # compute according to algorithm 10 in the paper
        print("Hyperparams: ", hyperparams, "Sample size: ", sample_num)
    plot(hyperparams, theoretical_sample_num) # draw the curve, where x-axis is the hyperparams and y-axis is the sample size
    plot(hyperparams, sample_num) # draw the curve, where x-axis is the hyperparams and y-axis is the sample size

In [1]:
import pandas as pd
import numpy as np
import config
import Algorithm10 as a10
from sklearn.tree import export_text

##### Config Variables #####
np = <module 'numpy' from '/opt/anaconda3/envs/learn/lib/python3.12/site-packages/numpy/__init__.py'>
dataset_path = ./dataset/Invistico_Airline.csv
model_path = ./models/
max_depth = 3
random_seed = 42
selected_features = ['Class', 'Seat comfort', 'Food and drink', 'Cleanliness', 'satisfaction']
rho = 0.85
alpha = 0.3
beta = 0.2
num_H = 10
m_up_bound = 43.091814768179134
tau_up_bound = 0.1107450928853292
tau = 0.011074509288532921
m = 100
############################


In [2]:
data = pd.read_csv('dataset/Invistico_Airline.csv')
data.head()

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0


In [3]:
data.shape

(129880, 22)

In [4]:
# function to check if two decision trees are equal
def are_trees_equal(tree1, tree2):
    # Check that both trees are fitted
    if not hasattr(tree1, 'tree_') or not hasattr(tree2, 'tree_'):
        raise ValueError("Both trees must be fitted before comparison.")

    # Compare parameters
    if tree1.get_params() != tree2.get_params():
        return False

    t1 = tree1.tree_
    t2 = tree2.tree_

    # Compare structure and splitting rules
    attributes_to_check = [
        'children_left', 'children_right',
        'feature', 'threshold',
        'impurity', 'n_node_samples', 'weighted_n_node_samples',
        'value'
    ]

    for attr in attributes_to_check:
        if not np.array_equal(getattr(t1, attr), getattr(t2, attr)):
            return False

    return True


In [5]:
def getConvergenceSampleNum(min_subset_size, max_subset_size, repeat_num, rho, sample_size_step=1):
    sample_size_replicablity_dict = {}
    
    X, y= a10.load_full_dataset(config.dataset_path, random_state=config.random_seed)
    for sample_size in range(min_subset_size, max_subset_size + 1, sample_size_step):
        #get dataset of size sample_size by sampling from the original dataset
        replicable_tree_list = []
        H = a10.build_candidate_trees(X, y,sample_size, max_depth=config.max_depth, num_trees=config.num_H, random_state=config.random_seed)    
        for i in range(repeat_num):
            print(f"sample size: {sample_size}, repeat: {i}")
            tree = a10.replicable_learner(X, y, H, sample_size, random_seed=config.random_seed+i)
            replicable_tree_list.append(tree)
            
        #check the probability if the trees in the replicable_tree_list are the same
        same_tree_count = 0
        for i in range(len(replicable_tree_list)):
            for j in range(i + 1, len(replicable_tree_list)):
                
                if are_trees_equal(replicable_tree_list[i], replicable_tree_list[j]):
                    # print(f"tree {i} and tree {j} are the same")
                    same_tree_count += 1
        prob = same_tree_count / (repeat_num * (repeat_num - 1) / 2)
        sample_size_replicablity_dict[sample_size] = prob
        if prob >= 1-rho:
            print(f"replicable at sample size: {sample_size}, prob: {prob}")
            return sample_size 
        
    # return sample_size_replicablity_dict
    print(f"not replicable at sample size between {min_subset_size} and {max_subset_size}, prob: {prob}")
    return -1

            
        
    

In [6]:
print("theoretical sample size: ", config.m_up_bound)
ans_dict = getConvergenceSampleNum(min_subset_size=10, max_subset_size=100, repeat_num=10, rho=config.rho, sample_size_step=10)

theoretical sample size:  43.091814768179134


100%|██████████| 10/10 [00:00<00:00, 149.35it/s]

sample size: 10, repeat: 0
OPT error 0.19999999999999996 errors dict_values([0.19999999999999996, 0.5, 0.5, 0.4, 0.30000000000000004, 0.7, 0.4, 0.5, 0.4, 0.5])
v: 0.20761117865681808
sample size: 10, repeat: 1
OPT error 0.0 errors dict_values([0.4, 0.0, 0.19999999999999996, 0.7, 0.19999999999999996, 0.5, 0.09999999999999998, 0.30000000000000004, 0.5, 0.30000000000000004])
v: 0.006174341076352328
sample size: 10, repeat: 2
OPT error 0.0 errors dict_values([0.19999999999999996, 0.4, 0.0, 0.09999999999999998, 0.19999999999999996, 0.19999999999999996, 0.30000000000000004, 0.5, 0.19999999999999996, 0.4])
v: 0.04338351607479349
sample size: 10, repeat: 3
OPT error 0.0 errors dict_values([0.19999999999999996, 0.8, 0.4, 0.0, 0.5, 0.30000000000000004, 0.19999999999999996, 0.5, 0.6, 0.4])
v: 0.03316268181755807
sample size: 10, repeat: 4


OPT error 0.0 errors dict_values([0.09999999999999998, 0.19999999999999996, 0.30000000000000004, 0.4, 0.0, 0.7, 0.30000000000000004, 0.30000000000000004, 0.19999999999999996, 0.30000000000000004])
v: 0.009877533969091901
sample size: 10, repeat: 5
OPT error 0.09999999999999998 errors dict_values([0.7, 0.5, 0.4, 0.6, 0.30000000000000004, 0.09999999999999998, 0.6, 0.30000000000000004, 0.19999999999999996, 0.30000000000000004])
v: 0.1283146877893961
sample size: 10, repeat: 6
OPT error 0.0 errors dict_values([0.5, 0.6, 0.4, 0.5, 0.30000000000000004, 0.5, 0.0, 0.6, 0.30000000000000004, 0.30000000000000004])
v: 0.049932139882341145
sample size: 10, repeat: 7
OPT error 0.09999999999999998 errors dict_values([0.5, 0.4, 0.6, 0.5, 0.4, 0.4, 0.5, 0.09999999999999998, 0.5, 0.4])
v: 0.10720377147521373
sample size: 10, repeat: 8
OPT error 0.19999999999999996 errors dict_values([0.4, 0.6, 0.4, 0.5, 0.4, 0.6, 0.30000000000000004, 0.6, 0.19999999999999996, 0.5])
v: 0.24149951776868242
sample size: 10

100%|██████████| 10/10 [00:00<00:00, 168.56it/s]


...

100%|██████████| 10/10 [00:00<00:00, 177.18it/s]


sample size: 70, repeat: 0
OPT error 0.22857142857142854 errors dict_values([0.27142857142857146, 0.2857142857142857, 0.30000000000000004, 0.22857142857142854, 0.2857142857142857, 0.2857142857142857, 0.30000000000000004, 0.24285714285714288, 0.37142857142857144, 0.2857142857142857])
v: 0.23618260722824666
sample size: 70, repeat: 1
OPT error 0.18571428571428572 errors dict_values([0.3142857142857143, 0.18571428571428572, 0.18571428571428572, 0.2571428571428571, 0.19999999999999996, 0.2857142857142857, 0.18571428571428572, 0.19999999999999996, 0.3142857142857143, 0.2571428571428571])
v: 0.19188862679063806
sample size: 70, repeat: 2
OPT error 0.15714285714285714 errors dict_values([0.34285714285714286, 0.17142857142857137, 0.15714285714285714, 0.2857142857142857, 0.30000000000000004, 0.19999999999999996, 0.15714285714285714, 0.2142857142857143, 0.2142857142857143, 0.3142857142857143])
v: 0.20052637321765063
sample size: 70, repeat: 3
OPT error 0.18571428571428572 errors dict_values([0.3

100%|██████████| 10/10 [00:00<00:00, 187.91it/s]

sample size: 80, repeat: 0


OPT error 0.23750000000000004 errors dict_values([0.2875, 0.3125, 0.3375, 0.25, 0.23750000000000004, 0.32499999999999996, 0.3125, 0.25, 0.30000000000000004, 0.2875])
v: 0.24511117865681817
sample size: 80, repeat: 1
OPT error 0.21250000000000002 errors dict_values([0.36250000000000004, 0.25, 0.36250000000000004, 0.25, 0.25, 0.30000000000000004, 0.21250000000000002, 0.21250000000000002, 0.32499999999999996, 0.30000000000000004])
v: 0.21867434107635236
sample size: 80, repeat: 2
OPT error 0.15000000000000002 errors dict_values([0.35, 0.22499999999999998, 0.19999999999999996, 0.25, 0.30000000000000004, 0.22499999999999998, 0.15000000000000002, 0.22499999999999998, 0.375, 0.275])
v: 0.1933835160747935
sample size: 80, repeat: 3
OPT error 0.1875 errors dict_values([0.275, 0.36250000000000004, 0.3125, 0.1875, 0.23750000000000004, 0.26249999999999996, 0.26249999999999996, 0.21250000000000002, 0.26249999999999996, 0.32499999999999996])
v: 0.22066268181755808
sample size: 80, repeat: 4
OPT erro

In [7]:
ans_dict

80